# Title

# Introduction

# Preliminary exploraty data analysis

In [1]:
### Loading Libraries into R
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [2]:
#reading data, all columns included, no need for skipping 
#data is tidy 
data <- read_csv("data/heart.csv")
data <- data|>
        mutate(HeartDisease = as.factor(HeartDisease))
data

Rows: 918 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope
dbl (7): Age, RestingBP, Cholesterol, FastingBS, MaxHR, Oldpeak, HeartDisease

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<fct>
40,M,ATA,140,289,0,Normal,172,N,0,Up,0
49,F,NAP,160,180,0,Normal,156,N,1,Flat,1
37,M,ATA,130,283,0,ST,98,N,0,Up,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1
38,M,NAP,138,175,0,Normal,173,N,0.0,Up,0


# Variable Explanation 

Age: age of the patient [years]

Sex: sex of the patient [M: Male, F: Female]

ChestPainType: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]

RestingBP: resting blood pressure [mm Hg]
Cholesterol: serum cholesterol [mm/dl]

FastingBS: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]

RestingECG: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]

MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]
ExerciseAngina: exercise-induced angina [Y: Yes, N: No]

Oldpeak: oldpeak = ST [Numeric value measured in depression]

ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]

HeartDisease: output class [1: heart disease, 0: Normal]

# These variables were excluded 

sex since we are interested to predict this for anyone regardless their sex and since this data only includes 2 sexes
 
we are only using numerical data to predict the heart disease so we have selected against other types of data coloumns (exept for the HeartDisease which we are interested in predicting)


In [3]:
# filter for ATA (Atypical Angina) type since this one is concerning and needed to be predicted for patients with this typ of pain

In [4]:
data_cleaned <- data|>
        filter(ChestPainType== "ATA")|>
        select(Age, RestingBP, Cholesterol, MaxHR, HeartDisease)
data_cleaned

Age,RestingBP,Cholesterol,MaxHR,HeartDisease
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
40,140,289,172,0
37,130,283,98,0
45,130,237,170,0
⋮,⋮,⋮,⋮,⋮
55,132,342,166,0
41,120,157,182,0
57,130,236,174,1


# Spliting data

In [5]:
### spliting the data into training set and testing set, 75% of the data in the training set
data_split <- initial_split(data_cleaned, prop = 0.75, strata = HeartDisease) 
data_train <- training(data_split)   
data_test <- testing(data_split)
data_train
data_test

Age,RestingBP,Cholesterol,MaxHR,HeartDisease
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
40,140,289,172,0
37,130,283,98,0
54,110,208,142,0
⋮,⋮,⋮,⋮,⋮
48,110,229,168,1
57,154,232,164,1
57,130,236,174,1


Age,RestingBP,Cholesterol,MaxHR,HeartDisease
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
45,130,237,170,0
39,120,204,145,0
54,120,273,150,0
⋮,⋮,⋮,⋮,⋮
58,120,284,160,1
44,120,263,173,0
35,122,192,174,0


In [6]:
#Tables 


diseased <- data_train|>
 #       mutate(na.rm_count =n(is.na()))|>
        group_by(HeartDisease)|>
        summarize(across(Age:MaxHR, max, na.rm=TRUE)) 
diseased

#normal <- data_train|>
   #     filter(HeartDisease == "0")|>
  #      group_by(HeartDisease)|>
 #       summarize(across(Age:MaxHR, max, na.rm=TRUE))
#normal
  #         Age = mean(diseased_Age, na.rm=TRUE),
   #         RestingBP = mean(diseased_RestinBP, na.rm=TRUE),
    #        Cholesterol = mean(diseased_Cholesterol, na.rm=TRUE),
     #       MaxHR = mean(diseased_MaxHR, na.rm=TRUE))



       #     Age = mean(normal_Age, na.rm=TRUE),
        #    RestingBP = mean(normal_RestinBP, na.rm=TRUE),
         #   Cholesterol = mean(normal_Cholesterol, na.rm=TRUE),
          #  MaxHR = mean(normal_MaxHR, na.rm=TRUE))


HeartDisease,Age,RestingBP,Cholesterol,MaxHR
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
0,74,190,468,202
1,74,192,319,195


# visalization 

In [9]:


#options()

data_hist <- ggplot(data_train, aes(x = Age, fill = as_factor(HeartDisease)))+
        geom_histogram(alpha=0.5, position="identity")+
        labs(x="a",y="b",fill="c")+
theme(text=element_text(size=12), scale_fill_discrete())

#facet_grid(rows=vars(HeartDisease))